## Geospatial Analysis Cookbook

SnowEx Hackweek  
July 13, 2021

David Shean

These are some common operations that you may need for your projects. We hope that you can review this code, and copy/paste to use in a modular fashion for your projects, so you don't have to reinvent the wheel when time is limited

## Importing csv of point data

## Load the existing csv into a Pandas DataFrame
* Use the relative path as in previous labs and the amazing Pandas `read_csv()` function
* Run a quick `head()` on your DataFrame to make sure everything looks right

In [ ]:
glas_fn = '../01_Shell_Github/data/GLAH14_tllz_conus_lulcfilt_demfilt.csv'
glas_df = pd.read_csv(glas_fn)

In [ ]:
glas_df.head()

## Convert the Pandas `DataFrame` to a GeoPandas `GeoDataFrame`
* See documentation here: https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html
* Careful about lon and lat order!
* Define coordinate reference system with crs keyword argument (4326 is geographic lat/lon on WGS84 Ellispoid) 
    * https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/intro-to-coordinate-reference-systems/
* Store in a variable named `glas_gdf` (needed for sample code later)
* Run a quick `head()` to make sure everything looks good
* You should have a new `geometry` column cointaining shapely Point objects

In [ ]:
glas_gdf = gpd.GeoDataFrame(glas_df, geometry=gpd.points_from_xy(glas_df['lon'], glas_df['lat']), crs='EPSG:4326')

# Save processed data to a GIS-ready file

The workflows in these Notebooks are intended to be fully reproducible, starting with raw data and producing all final output.  But sometimes you want to write out geospatial data for analysis in a GUI-based GIS (QGIS, ArcMap), or to share with colleagues who will use these tools.

## Check available output formats for geopandas
* Use fiona to get a list of available file type drivers for output
* Note: the 'r' means fiona/geopandas can read this file type, 'w' means it can write this file type, 'a' means it can append to an existing file.
    * https://fiona.readthedocs.io/en/latest/manual.html#writing-vector-data

In [ ]:
import fiona
fiona.supported_drivers

## How to choose a format?
* If you've taken a GIS class (or not), you've probably used shapefiles in the past.  Please stop.  The ESRI shapefile is a legacy format, though it is still widely used.
* http://switchfromshapefile.org/
* Better options these days are Geopackage (GPKG) when spatial index is required, and simple GeoJSON for vector data in EPSG:4326
    * Both should be supported by your GIS (including QGIS, ArcGIS, etc)
* Let's use geopackage for this exercise
* Can use the Geopandas `to_file()` method to create this file
    * Make sure you properly specify filename with extension and the `driver` option
    * *Note: Writing out may take a minute, and may produce an intermediate '.gpkg-journal' file*
        * Can see this in the file browser or terminal!

In [ ]:
out_fn='./conus_glas_gdf_aea_rgi_agg.gpkg'
glas_gdf_aea_rgi_agg_gdf.to_file(out_fn, driver='GPKG')

#out_fn='./conus_glas_gdf_aea_rgi_agg.geojson'
#glas_gdf_aea_rgi_agg_gdf.to_file(out_fn, driver='GeoJSON')

In [ ]:
ls -lh $out_fn

## 🎉

You can now directly load this gpkg file in any GIS, without defining a coordinate system. You can also load this file directly into geopandas in the future using the `read_file()` method, without having to repeat the processing above.

### See for yourself!
Try it! Right-click on file in the file browser to the left of the JupyterLab interface, then select Download and pick a location on your local computer (e.g., your Downloads folder). 

Then open this file in QGIS or ArcGIS on your local machine!

# USGS 3DEP Raster Tile Download - Mesa County LiDAR

In [ ]:
import os
import rasterio as rio
import rasterio.plot
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Current url for 1/9th arcsec 3DEP raster products
#url1='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/19/IMG/ned19_n48x25_w122x50_wa_puget_sound_2000.zip'
#fn1='ned19_n48x25_w122x50_wa_puget_sound_2000.img'
#Going to fetch and read zipfile on the fly, specifying img inside
#path=f'/vsizip//vsicurl/{url1}/{fn1}'
#os.path.splitext(os.path.split(url_list[0])[-1])[0]+'.img'

In [ ]:
#GUI based search for tiles
#https://prd-tnm.s3.amazonaws.com/LidarExplorer/index.html
#Contains public url for each tif
url_fn_3DEP = 'gm_3dep_1m_lidar_tiles.txt'

In [ ]:
with open(url_fn_3DEP) as f:
    url_list = f.read().splitlines()

In [ ]:
url_list.sort()

In [ ]:
#idx = ['UTM13' in i for i in url_list]

In [ ]:
path_list = []
for url in url_list:
    #fn = os.path.splitext(os.path.split(url)[-1])[0]+'.img'
    #path = f'/vsizip//vsicurl/{url}/{fn}'
    path = f'/vsicurl/{url}'
    path_list.append(path)
path_list_str = ' '.join(path_list)

In [ ]:
path

In [ ]:
with rio.open(path) as src:
    rio.plot.show(src)

In [ ]:
url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1m/Projects/CO_MesaCo_QL2_UTM12_2016/TIFF/USGS_one_meter_x74y433_CO_MesaCo_QL2_UTM12_2016.tif'

In [ ]:
path = f'/vsicurl/{url}'

In [ ]:
with rio.open(path) as src:
    rio.plot.show(src)

In [ ]:
vrt_fn = os.path.splitext(url_fn_3DEP)[0]+'.vrt'
tif_fn = os.path.splitext(url_fn_3DEP)[0]+'.tif'
hs_fn = os.path.splitext(tif_fn)[0]+'_hs.tif'

In [ ]:
#This actually takes some time as file must be downloaded and unzipped to read img header
!gdalbuildvrt $vrt_fn $path_list_str

In [ ]:
dst_crs = 'EPSG:32612'

In [ ]:
#Since these tiles are mixed projection, can download, reproject and mosaic in one go
!gdalwarp -r cubic -tr 3.0 3.0 -dstnodata -9999 -t_srs $dst_crs \
-co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER \
$path_list_str $tif_fn

In [ ]:
!gdaldem hillshade -co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER $tif_fn $hs_fn

In [ ]:
!gdaladdo -r gauss $hs_fn

In [ ]:
src = rio.open(hs_fn, overview_level=1)

In [ ]:
src.profile

In [ ]:
rio.plot.show(src, cmap='gray')

# Retrieve Preliminary Feb 2017 Grand Mesa WorldView Stereo Snow Depth data product from Zenodo repository
* Snow depth raster derived from WorldView-3 DEMs for comparisons in McGrath et al. (2019) against ASO lidar snow depth, GPR snow depth and snow pit data
* https://doi.org/10.5281/zenodo.3381652

In [ ]:
# https://zenodo.org/record/3381653
url = 'https://zenodo.org/record/3381653/files/20160925_gm_8m_trans-tile-0_20170201_gm_8m_trans-tile-0_dz_eul.tif'

In [ ]:
#This uses the GDAL /vsicurl_streaming approach
test_fn = f'/vsicurl_streaming/{url}'

In [ ]:
src = rio.open(test_fn)

In [ ]:
src.profile

In [ ]:
a = src.read(1, masked=True)

In [ ]:
f, ax = plt.subplots(figsize=(10,10))
ax.imshow(a, cmap='inferno', clim=(0, 2), interpolation='none', extent=rio.plot.plotting_extent(src))

## Function to add day of year and day of water year to dataframe

In [ ]:
#Add DOY and DOWY column
def add_dowy(df, col=None):
    print("Adding Day of Water Year (DOWY)")
    if col is None:
        df['doy'] = df.index.dayofyear
    else:
        df['doy'] = pd.to_datetime(df[col]).dt.dayofyear
    
    #df['dowy'] = (df['doy'].index - pd.DateOffset(months=9)).dayofyear
    # Sept 30 is doy 273
    df['dowy'] = df['doy'] - 273
    df.loc[df['dowy'] <= 0, 'dowy'] += 365
    df['decyear'] = Time(pd.to_datetime(df[col])).decimalyear

## Clean up ASO DTM and Snow Depth data from NSIDC

In [ ]:
aso_dir = '/nobackupp8/deshean/data/ASO'
aso_fn_list = !ls $aso_dir/ASO*SD*tif $aso_dir/ASO*DTM*tif
aso_fn_list

## Prepare polygon masks around valid data in raster
* This is a common task to generate polygons around valid pixels in a raster
* Can do this on a low-resolution overview (8x subsampling) for efficiency

In [ ]:
aso_feat_list = []
ovr_scale = 8
for fn in aso_fn_list:
    src = rio.open(fn)
    a = src.read(1, masked=True, out_shape=(src.count, int(src.height / ovr_scale), int(src.width / ovr_scale)))
    ovr_transform = src.transform * src.transform.scale(src.width / a.shape[-1], src.height / a.shape[-2])
    mask = np.ma.getmaskarray(a).astype(np.uint8) - 1
    shapes = features.shapes(mask, mask=mask, transform=ovr_transform)
    geom_list = list(shapes)
    dt = os.path.splitext(os.path.split(fn)[-1])[0].split('_')[-1]
    aso_feat_list.extend([{'properties':{'fn':fn, 'date':dt}, 'geometry':i[0]} for i in geom_list])

In [ ]:
aso_gdf = gpd.GeoDataFrame.from_features(aso_feat_list, crs=src.crs)
aso_gdf = aso_gdf.to_crs(crs)

In [ ]:
aso_gdf.plot(alpha=0.1)

In [ ]:
#remove tiny polygons
area_thresh = 1e8
aso_gdf = aso_gdf.loc[aso_gdf.area > area_thresh]

In [ ]:
aso_gdf.plot(alpha=0.1, edgecolor='k', legend=True)

In [ ]:
#Shoud simplify and sieve
#aso.geometry.simplify(tolerance=100)

In [ ]:
aso_gdf['date'] = pd.to_datetime(aso_gdf['date'])

In [ ]:
add_dowy(aso_gdf, col='date')

In [ ]:
aso_gdf['platform'] = 'ASO'
aso_gdf['acqdate'] = aso_gdf['date']

In [ ]:
aso_gdf.set_index('date', inplace=True)

In [ ]:
aso_gdf

# Strategies for Dynamic DEM Data Download and Processing

In [ ]:
import os
import requests
import rasterio as rio
from rasterio import plot, mask

In [ ]:
gm_bounds = [-108.34115668,   38.82320553, -107.72839859,   39.19563035]

## Use OpenTopography GlobalDEM API to fetch clipped DEM
* OpenTopgraphy is a fantastic organization that "facilitates community access to high-resolution, Earth science-oriented, topography data, and related tools and resources."
    * https://opentopography.org/about
* One of the many services they provide is an API for several popular Global DEM datasets, with simple subsetting and delivery: https://opentopography.org/developers
* Latest list of available DEM: https://portal.opentopography.org/apidocs/#/Public/getGlobalDem
* We'll use this service to extract a small portion of the SRTM-GL3 DEM

In [ ]:
#List of all products hosted by OpenTopography GlobalDEM API
demtype_list = ["SRTMGL3", "SRTMGL1", "SRTMGL1_E", "AW3D30", "AW3D30_E", "SRTM15Plus"]

In [ ]:
#1 arcsec
demtype = "SRTMGL1_E"

In [ ]:
base_url="https://portal.opentopography.org/API/globaldem?demtype={}&west={}&south={}&east={}&north={}&outputFormat=GTiff"

In [ ]:
base_url.format(demtype, *gm_bounds)

### Optional: set up API key to access Copernicus DEM and NASADEM
* This will enable access to NASADEM and Compernicus DEM (30 and 90 m)
* https://portal.opentopography.org/myopentopo

In [ ]:
#Paste API key below
api_key = None
if api_key:
    demtype_list.extend(['NASADEM', 'COP30', 'COP90'])
    demtype = 'COP30'

In [ ]:
def get_OT_GlobalDEM(demtype, bounds, out_fn=None, api_key=None):
    if out_fn is None:
        out_fn = '{}.tif'.format(demtype)
    
    if not os.path.exists(out_fn):
        #Prepare API request url
        #Bounds should be [minlon, minlat, maxlon, maxlat]
        url = base_url.format(demtype, *bounds)
        if api_key is not None:
            url = f'{url}&API_Key={api_key}'
        print(url)
        #Get
        response = requests.get(url)
        #Should check for 200
        #Write to disk
        open(out_fn, 'wb').write(response.content)

In [ ]:
out_fn = f"GM_{demtype}.tif"
out_fn

In [ ]:
get_OT_GlobalDEM(demtype, gm_bounds, out_fn, api_key)

In [ ]:
!ls -lh $out_fn

In [ ]:
with rio.open(out_fn) as src:
    rio.plot.show(src)

### Reproject points to match raster

In [ ]:
glas_gdf_srtm = glas_gdf_aea.to_crs(srtm_src.crs)

### Prepare the coordinate arrays to pass to rio `sample` function
* The `sample` function expects a list of (x,y) tuples: https://rasterio.readthedocs.io/en/latest/api/rasterio.sample.html
    * Need to create this from the `geometry` objects in your GeoDataFrame
    * You want a list of the form [(x1,y1),(x2,y2),…]
* Pass to `sample`
* Note that the `sample` function returns a `generator` object, and it doesn't actually evaluate the call!
* Can wrap this in a `np.array(list())` to evaluate, or use `np.fromiter()`
* This operation may take ~10-20 seconds to complete

In [ ]:
glas_coord = [(pt.x, pt.y) for pt in glas_gdf_srtm.geometry]
#glas_coord = np.vstack((glas_gdf_srtm.geometry.x.values, glas_gdf_srtm.geometry.y.values)).T

### Sample with rasterio

In [ ]:
glas_srtm_sample = srtm_src.sample(glas_coord)
glas_srtm_sample

### This is a generator, so we actually need to evaluate

In [ ]:
glas_srtm_elev = np.fromiter(glas_srtm_sample, dtype=srtm.dtype)
glas_srtm_elev

### Deal with nodata
* Some of our GLAS points are located over areas where we don't have valid DEM pixels
* These will be assigned the raster nodata value (-32768 in this case)

In [ ]:
glas_srtm_elev_ma = np.ma.masked_equal(glas_srtm_elev, srtm_src.nodata)
glas_srtm_elev_ma

### Add new column to the GeoDataFrame
* Set masked values to `np.nan` (which requires a conversion to float)

In [ ]:
glas_gdf_srtm['srtm_90m_z_rio'] = glas_srtm_elev_ma.astype(float).filled(np.nan)

In [ ]:
glas_gdf_srtm.dropna().head()

In [ ]:
f, ax = plt.subplots()
ax.imshow(hs, cmap='gray', extent=rio.plot.plotting_extent(srtm_hs_src))
#ax.imshow(srtm, extent=srtm_extent, alpha=0.5);
glas_gdf_srtm.dropna().plot('srtm_90m_z_rio', ax=ax, markersize=1);

*Note: the SRTM elevation values are height above the EGM96 geoid*

### Notes on sampling coarse rasters or noisy rasters at integer pixel locations
* The rasterio approach is efficient, but it uses a nearest neighbor algorithm to extract the elevation value for the grid cell that contains the point, regardless of where the point falls within the grid cell (center vs. corner)
* But the DEM grid cells can be big (~90x90 m for the SRTM3 data), so if point is near the corner of a pixel on steep slope, the pixel value might not be representative.
* A better approach is to use bilinear or bicubic sampling, to interpolate the elevation value at the point coordinate using pixel values within some neighborhood around the point, (e.g. 2x2 window for bilinear, 4x4 window for cubic)
* Other approaches involve computing zonal stats within some radius of the point location (e.g., median elevation of pixels within 300 m of the point, using buffer to create polygons)
    * https://www.earthdatascience.org/courses/earth-analytics-python/lidar-remote-sensing-uncertainty/extract-data-from-raster/
    * https://pysal.org/scipy2019-intermediate-gds/deterministic/gds2-rasters.html#getting-values-at-cells
    * https://github.com/dshean/pygeotools/blob/master/pygeotools/lib/geolib.py#L1019

## 2. Local window sample

https://github.com/dshean/demcoreg/blob/master/demcoreg/sample_raster_at_pts.py

https://github.com/dshean/pygeotools/blob/master/pygeotools/lib/geolib.py#L1019

## 3. Scipy ndimage: n-order polynomial
* Good option for regular grids (i.e., DEMs)
* Propagates nan, issues when DEM has missing data

In [ ]:
import scipy.ndimage
#Should dropna here
yx = np.array([glas_gdf_srtm.geometry.y, glas_gdf_srtm.geometry.x])
#Convert map coordinates to array coordinates (want float, not whole integer indices)
#See functions here
#https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html
#Need to revisit
#Use bilinear resampling here (order=1)
ndimage_samp = scipy.ndimage.map_coordinates(srtm, yx, order=1, mode='nearest')
ndimage_samp = np.ma.masked_equal(ndimage_samp, srtm_src.nodata)

In [ ]:
srtm_src.transform

In [ ]:
glas_gdf_srtm['srtm_90m_z_ndimage'] = ndimage_samp.astype(float).filled(np.nan)

In [ ]:
glas_gdf_srtm.dropna().head()

## Handling complex horizontal and vertical datum transformations
* https://pyproj4.github.io/pyproj/stable/build_crs.html

In [ ]:
import pyproj
from pyproj.crs import CRS, CompoundCRS

## 2D WGS84 coordinates (lat, lon)

In [ ]:
CRS(4326)

## 3D WGS84 coordinates (lat, lon, height above ellipsoid)

In [ ]:
CRS(4979)

In [ ]:
CRS(7789)

In [ ]:
CRS("EPSG:32610").to_3d()

In [ ]:
CRS("EPSG:32610").to_3d()

In [ ]:
cmpd_crs = CompoundCRS(name="NAD83 + NAVD88", components=["EPSG:4269", "EPSG:5703"])
cmpd_crs = CRS("EPSG:4269+5703")
#cmpd_crs = CompoundCRS(name="WGS 84 + EGM96", components=["EPSG:4326", "EPSG:5773"])
#cmpd_crs = CRS("EPSG:4326+5773")

In [ ]:
cmpd_crs

In [ ]:
gm_centroid = (39.02687, -108.06352, 0)

In [ ]:
#(y, x, z, t)
snotel_coord = (39.05831, -108.05835, 3048, 2010.0)

In [ ]:
trans = pyproj.Transformer.from_crs(cmpd_crs, "EPSG:4979")
trans.transform(*snotel_coord)

In [ ]:
trans = pyproj.Transformer.from_crs(cmpd_crs, "EPSG:4979")
trans.transform(*snotel_coord)

In [ ]:
trans = pyproj.Transformer.from_crs(cmpd_crs, "EPSG:32612")
trans.transform(*snotel_coord)

In [ ]:
trans = pyproj.Transformer.from_crs(cmpd_crs, "EPSG:26912")
trans.transform(*snotel_coord)

In [ ]:
trans = pyproj.Transformer.from_crs(cmpd_crs, "EPSG:26912+5703")
trans.transform(*snotel_coord)

In [ ]:
CRS(26912)

In [ ]:
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

aoi = AreaOfInterest(west_lon_degree=gm_centroid[1], 
                     south_lat_degree=gm_centroid[0],
                     east_lon_degree=gm_centroid[1],
                     north_lat_degree=gm_centroid[0])

utm_crs_list = query_utm_crs_info(datum_name="NAD83", area_of_interest=aoi)

utm_crs = CRS.from_epsg(utm_crs_list[0].code)

In [ ]:
utm_crs

## Vertical conversion using PROJ and GDAL

In [ ]:
Attempts to convert from NAD83+NAVD88 to WGS84+ellipsoid

In [ ]:
gdalwarp -overwrite -s_srs EPSG:26912+5703 -t_srs '+proj=utm +zone=12 +datum=WGS84 +units=m +no_defs +type=crs +grids=us_noaa_g2018u0.tif' USGS_one_meter_
x74y432_CO_MesaCo_QL2_UTM12_2016.tif test.tif

In [ ]:
gdalwarp -overwrite -s_srs 'EPSG:26912+5703' -t_srs 'EPSG:32612+4979' USGS_one_meter_x74y432_CO_MesaCo_QL2_UTM12_2016.tif test3.tif